In [1]:
%matplotlib widget
import numpy as np
import os
import h5py
import subprocess

print(os.getpid())
%cd ../
import pylib.mix as mix
%cd "./jupyter-notebooks/QSVT-QSP-angles-for-matlab-c#/Hamiltonian/QSP/"

11859
/media/work/docs/codes/QuCF/scripts-py
/media/work/docs/codes/QuCF/scripts-py/jupyter-notebooks/QSVT-QSP-angles-for-matlab-c#/Hamiltonian/QSP


In [2]:
mix.reload_module(mix)

# -----------------------------------------------------------
def find_exp(number) -> int:
    base10 = np.log10(abs(number))

    temp = np.floor(base10)
    flag_neg = False
    if temp < 0:
        flag_neg = True

    return abs(temp), flag_neg
# -----------------------------------------------------------
def txt_to_hdf5(old_filename, new_filename):
    function_type_res = "QSP-ham"
    rescaling_factor_res = 1.
    parity_res = -1

    # --- read the .angles (text) file ---
    print("read angles from:\n " + old_filename)
    with open(old_filename) as f:
        param_res = float(f.readline()[:-1]) 
        eps_res   = float(f.readline()[:-1]) 
        Na        = int(f.readline()[:-1]) 
        angles = np.zeros(Na)
        for ii in range(Na):
            angles[ii] = float(f.readline())
            
    print()
    print("function type: \t\t{:s}".format(function_type_res))    
    print("rescaling factor: \t{:0.3f}".format(rescaling_factor_res))
    print("parity: \t\t{:d}".format(parity_res))
    print("parameter: \t\t{:0.3e}".format(param_res))
    print("error: \t\t\t{:0.3e}".format(eps_res))
    print("N-angles: \t\t{:d}".format(Na))
    print()       

    # --- write the .hdf5 file ---
    print("write angles to:\n " + new_filename)
    with h5py.File(new_filename, "w") as f:
        grp = f.create_group("basic")
        
        grp.create_dataset('polynomial_type',  data=function_type_res)
        grp.create_dataset('rescaling_factor', data=float(rescaling_factor_res))
        grp.create_dataset('parity',           data=parity_res)
        grp.create_dataset('eps',              data=eps_res)
        grp.create_dataset('par',              data=param_res)

        grp = f.create_group("angles")
        grp.create_dataset('QSP-ham',  data = angles)
    return

In [3]:
# ----------------------------------------------------------
# --- Specify QSP parameters ---
# ----------------------------------------------------------
t_moments = [1.244745728791e+03]

Nt = len(t_moments)
eps_QSP_array = [1e-12] * Nt

case_format = 1

# --- Form the name of the output name ---
file_names = [None] * Nt
print("--- Chosen QSP parameters ---")
if case_format == 1:
    for it in range(Nt):
        eps_QSP = eps_QSP_array[it]
        t1 = t_moments[it]
        exp_t1, flag_neg = find_exp(t1)

        str_exp = "p"
        if flag_neg:
            str_exp = "m"

        if flag_neg:
            temp_int = int(t1 * 10**(exp_t1+3))
        else:
            temp_int = int(t1 / 10**(exp_t1-3))

        file_names[it] = "QSP_t{:d}{:s}{:d}_eps{:d}".format(
            temp_int, 
            str_exp,
            int(exp_t1),
            int(abs(np.round(np.log10(eps_QSP))))
        )

        print("t, eps: {:0.3e}, {:0.3e}".format(t1, eps_QSP))
        print("file name: {:s}".format(file_names[it]))
        print()

--- Chosen QSP parameters ---
t, eps: 1.245e+03, 1.000e-12
file name: QSP_t1244p3_eps12



In [4]:
# ------------------------------------------------
# --- Compute QSP angles ---
# ------------------------------------------------
line_code = "/media/work/docs/codes/progs/Q#F#/test-qsp/bin/Debug/netcoreapp3.1/qsp"
for it in range(Nt):
    eps_QSP = eps_QSP_array[it]
    t1 = t_moments[it]

    line_cmd = "{:s} {:0.3e} {:0.3e} {:s} {:s}".format(
        line_code, t1, eps_QSP, "./", file_names[it]
    )
    subprocess.run(line_cmd, shell = True, executable="/bin/bash")

    # from the .txt file to .hdf5 file:
    txt_to_hdf5(file_names[it], file_names[it] + ".hdf5")

    # remove the .txt file:
    line_cmd = "rm {:s}".format(file_names[it])
    subprocess.run(line_cmd, shell = True, executable="/bin/bash")

Start calculating...
Chosen tau = 1245
Chosen eps = 1E-12
polyA = { Denominator = 9007199254740992
  Numerator =
             2z^-1358 + 4z^-1356 + 11z^-1354 + 25z^-1352 + 57z^-1350 + 130z^-1348 + 293z^-1346 + 655z^-1344 + 1448z^-1342 + 3178z^-1340 + 6918z^-1338 + 14937z^-1336 + 31992z^-1334 + 67957z^-1332 + 143156z^-1330 + 299041z^-1328 + 619371z^-1326 + 1271823z^-1324 + 2588886z^-1322 + 5223490z^-1320 + 10445297z^-1318 + 20698622z^-1316 + 40641439z^-1314 + 79058406z^-1312 + 152342259z^-1310 + 290754011z^-1308 + 549542195z^-1306 + 1028443936z^-1304 + 1905441849z^-1302 + 3494413583z^-1300 + 6342215402z^-1298 + 11389811437z^-1296 + 20235672652z^-1294 + 35559576642z^-1292 + 61793210614z^-1290 + 106162784426z^-1288 + 180279982581z^-1286 + 302521528777z^-1284 + 501512164178z^-1282 + 821103780964z^-1280 + 1327309740599z^-1278 + 2117681478548z^-1276 + 3333562565241z^-1274 + 5175463683793z^-1272 + 7921392826079z^-1270 + 11947258542217z^-1268 + 17747353173393z^-1266 + 25951361181124z^-1264 + 3